## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.11,65,75,16.11,broken clouds
1,1,Margate,GB,51.3813,1.3862,62.31,80,100,4.00,overcast clouds
2,2,Haines Junction,CA,60.7522,-137.5108,46.35,57,99,4.61,overcast clouds
3,3,Erenhot,CN,43.6475,111.9767,48.72,29,49,11.23,scattered clouds
4,4,Coaldale,CA,49.7167,-112.6185,67.32,41,20,14.97,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_city_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
7,7,Alyangula,AU,-13.8483,136.4192,87.67,62,40,10.36,scattered clouds
19,19,Cabo San Lucas,MX,22.8909,-109.9124,87.51,65,1,6.91,clear sky
22,22,Airai,TL,-8.9266,125.4092,79.41,26,44,3.00,scattered clouds
26,26,Victoria,HK,22.2855,114.1577,89.65,80,0,1.99,clear sky
27,27,Ojinaga,MX,29.5667,-104.4167,82.42,39,75,17.27,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.notnull().sum()

City_ID        228
City           228
Country        228
Lat            228
Lng            228
Max Temp       228
Humidity       228
Cloudiness     228
Wind Speed     228
Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# There are no empty rows, therefore

clean_df = preferred_city_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Alyangula,AU,87.67,scattered clouds,-13.8483,136.4192,
19,Cabo San Lucas,MX,87.51,clear sky,22.8909,-109.9124,
22,Airai,TL,79.41,scattered clouds,-8.9266,125.4092,
26,Victoria,HK,89.65,clear sky,22.2855,114.1577,
27,Ojinaga,MX,82.42,broken clouds,29.5667,-104.4167,
30,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
31,San Rafael Del Sur,NI,77.97,clear sky,11.8485,-86.4384,
32,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
33,Tomigusuku,JP,85.77,few clouds,26.1850,127.6750,
39,Labuhan,ID,84.70,overcast clouds,-6.8844,112.2051,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
          hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
          print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= ""]
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Alyangula,AU,87.67,scattered clouds,-13.8483,136.4192,Groote Eylandt Lodge
19,Cabo San Lucas,MX,87.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
22,Airai,TL,79.41,scattered clouds,-8.9266,125.4092,Juvinal Martin
26,Victoria,HK,89.65,clear sky,22.2855,114.1577,Mini Hotel Central
27,Ojinaga,MX,82.42,broken clouds,29.5667,-104.4167,Suites Las Palmas
...,...,...,...,...,...,...,...
665,Houma,US,79.00,overcast clouds,29.5958,-90.7195,Courtyard by Marriott Houma
678,Mackay,AU,83.68,broken clouds,-21.1500,149.2000,International Lodge Motel
679,Oda,JP,77.77,broken clouds,35.1833,132.5000,Sky Hotel Oda
680,Soyo,AO,75.27,overcast clouds,-6.1349,12.3689,Hotel Mpampa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacations.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info,)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))